<a href="https://colab.research.google.com/github/YuichH/ML/blob/main/Kaggle/ZillowPrize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#共通


In [ ]:
import sys
# Google colab環境であるか判定
if 'google.colab' in sys.modules:
    # マウントを行う
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Not Google Colab")

Mounted at /content/drive


In [ ]:
# Google Colab. やJupyter Lab.でプロットするためには，以下を実行する．
import plotly.io as pio
pio.renderers.default = "colab"

#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import numpy as np
from scipy.stats import norm
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
%matplotlib inline

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

from subprocess import check_output
import datetime as dt
import gc

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#パラメータ
import argparse
parser = argparse.ArgumentParser()
# parser.add_argument("--n_epoch", type=int, default=100)
# parser.add_argument("--batch_size", type=int, default=128)
# parser.add_argument("--lr", type=float, default=0.001)
# parser.add_argument("--model_name", default="Zillow Prize")
parser.add_argument("--work_dir", default="/content/drive/MyDrive/Kaggle/Zillow Prize/")
parser.add_argument("--input_dir", default="/content/drive/MyDrive/Kaggle/Zillow Prize/")
# parser.add_argument("--model_dir", default="/content/drive/MyDrive/Kaggle/Zillow Prize/")
opt = parser.parse_args(args=[])
print(opt)

Namespace(work_dir='/content/drive/MyDrive/Kaggle/Zillow Prize/', input_dir='/content/drive/MyDrive/Kaggle/Zillow Prize/')


#Zillow Prize: Zillow’s Home Value Prediction (Zestimate)
https://www.kaggle.com/competitions/zillow-prize-1/overview

各プロパティ（ユニークなパーセルID）に対して、各タイムポイントの対数誤差を予測する必要があります。あなたは、6つのタイムポイントを予測する必要があります：2016年10月（201610）、2016年11月（201611）、2016年12月（201612）、2017年10月（201710）、2017年11月（201711）、2017年12月（201712）です。

logerror = log(zestimate)-log(saleprice)

ファイルの説明<br>
properties_2016.csv - 2016年のホーム機能を持つ全物件。注：2017年の新規物件の中には、parcelid'sを除いてまだデータを持っていないものがあります。これらのデータポイントは、properties_2017.csvが利用可能になったときに入力されるはずです。<br>
properties_2017.csv - 2017年のホーム機能を持つすべての物件（2017/10/2にリリースされました。）<br>
train_2016.csv - 2016/1/1から2016/12/31までのトランザクションを含むトレーニングセット<br>
train_2017.csv - 2017/1/1～2017/9/15の取引を含むトレーニングセット（2017/10/2リリース）<br>
sample_submission.csv - 正しい形式のサンプル提出ファイルです。<br>

##How to Compete for Zillow Prize at Kaggle
https://www.kaggle.com/competitions/zillow-prize-1/discussion/39578

In [ ]:
properties = pd.read_csv(opt.input_dir + "properties_2016.csv")
properties.columns

Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildin

In [ ]:
properties.head(5)

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.000,0.000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.000,2015.000,9.000,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.000,0.000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.000,2015.000,27516.000,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.000,0.000,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.000,1413387.000,2015.000,762631.000,20800.370,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.000,0.000,3.000,7.000,NaN,NaN,...,1.000,NaN,571346.000,1156834.000,2015.000,585488.000,14557.570,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.000,0.000,4.000,NaN,NaN,NaN,...,NaN,NaN,193796.000,433491.000,2015.000,239695.000,5725.170,NaN,NaN,NaN


In [ ]:
train = pd.read_csv(opt.input_dir + "train_2016_v2.csv")
train.columns

Index(['parcelid', 'logerror', 'transactiondate'], dtype='object')

In [ ]:
train.head(5)

,parcelid,logerror,transactiondate
0,11016594,0.028,2016-01-01
1,14366692,-0.168,2016-01-01
2,12098116,-0.004,2016-01-01
3,12643413,0.022,2016-01-02
4,14432541,-0.005,2016-01-02


In [ ]:
test = pd.read_csv(opt.input_dir + "sample_submission.csv")
test.columns

Index(['ParcelId', '201610', '201611', '201612', '201710', '201711', '201712'], dtype='object')

In [ ]:
test= test.rename(columns={'ParcelId': 'parcelid'}) #To make it easier for merging datasets on same column_id later

In [ ]:
test.head(5)

,parcelid,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [ ]:
print("Training Size:" + str(train.shape))
print("Property Size:" + str(properties.shape))
print("Sample Size:" + str(test.shape))

Training Size:(90275, 3)
Property Size:(2985217, 58)
Sample Size:(2985217, 7)


##前処理

In [ ]:
# int/float64⇒int/float32
for c, dtype in zip(properties.columns, properties.dtypes):
    if dtype == np.float64:
        properties[c] = properties[c].astype(np.float32)
    if dtype == np.int64:
        properties[c] = properties[c].astype(np.int32)

for column in test.columns:
    if test[column].dtype == int:
        test[column] = test[column].astype(np.int32)
    if test[column].dtype == float:
        test[column] = test[column].astype(np.float32)

In [ ]:
#living area proportions
properties['living_area_prop'] = properties['calculatedfinishedsquarefeet'] / properties['lotsizesquarefeet']
#tax value ratio
properties['value_ratio'] = properties['taxvaluedollarcnt'] / properties['taxamount']
#tax value proportions
properties['value_prop'] = properties['structuretaxvaluedollarcnt'] / properties['landtaxvaluedollarcnt']

In [ ]:
# merging the properties dataset with training and testing dataset for model building and testing prediction
df_train = train.merge(properties, how='left', on='parcelid')
df_test = test.merge(properties, how='left', on='parcelid')

In [ ]:
### Remove previos variables to keep some memory
del properties, train
gc.collect();

print('Memory usage reduction…')
df_train[['latitude', 'longitude']] /= 1e6
df_test[['latitude', 'longitude']] /= 1e6

df_train['censustractandblock'] /= 1e12
df_test['censustractandblock'] /= 1e12

Memory usage reduction…


In [ ]:
# Label Encoding For Machine Learning & Filling Missing Values
from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()
for c in df_train.columns:
    df_train[c]=df_train[c].fillna(0)
    if df_train[c].dtype == 'object':
        lbl.fit(list(df_train[c].values))
        df_train[c] = lbl.transform(list(df_train[c].values))

for c in df_test.columns:
    df_test[c]=df_test[c].fillna(0)
    if df_test[c].dtype == 'object':
        lbl.fit(list(df_test[c].values))
        df_test[c] = lbl.transform(list(df_test[c].values))

In [ ]:
# We will now drop the features that serve no useful purpose.
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
x_test = df_test.drop(['parcelid', 'propertyzoningdesc', 'propertycountylandusecode', '201610', '201611', '201612', '201710', '201711', '201712'], axis = 1)

In [ ]:
x_train = x_train.values
y_train = df_train['logerror'].values

In [ ]:
print(x_train[0])
print(y_train[0])

[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  2.00000000e+00
  3.00000000e+00  0.00000000e+00  4.00000000e+00  2.00000000e+00
  0.00000000e+00  0.00000000e+00  1.68400000e+03  1.68400000e+03
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.03700000e+03  0.00000000e+00  2.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  2.00000000e+00  3.42809906e+01
 -1.18488533e+02  7.52800000e+03  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  2.61000000e+02
  6.03710680e+07  1.24470000e+04  3.10100000e+03  3.18170000e+04
  9.63700000e+04  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  1.95900000e+03  0.00000000e+00  0.00000000e+00  1.22754000e+05
  3.60170000e+05  2.01500000e+03  2.37416000e+05  6.73587988e+03
  0.00000000e+00  0.00000000e+00  6.03710670e+01  2.23698199e-01
  5.34703712e+01  5.17041802e-01]
0.0276


##Cross Validation


In [ ]:
from sklearn.model_selection import train_test_split
X = x_train
y = y_train

Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Implement the Xgboost
import xgboost as xgb
dtrain = xgb.DMatrix(Xtrain, label=ytrain)
dvalid = xgb.DMatrix(Xvalid, label=yvalid)
dtest = xgb.DMatrix(x_test.values)

In [ ]:
# Try different parameters!
xgb_params = {'min_child_weight': 5, 'eta': 0.035, 'colsample_bytree': 0.5, 'max_depth': 4, 'subsample': 0.85, 'lambda': 0.8, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1, 'gamma' : 0, 'eval_metric': 'mae', 'objective': 'reg:linear' }
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

model_xgb = xgb.train(xgb_params, dtrain, 1000, watchlist, early_stopping_rounds=100, maximize=False, verbose_eval=10)

[23:37:21] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[23:37:21] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-mae:0.47988	valid-mae:0.48093
[10]	train-mae:0.34206	valid-mae:0.34313
[20]	train-mae:0.24750	valid-mae:0.24855
[30]	train-mae:0.18325	valid-mae:0.18422
[40]	train-mae:0.14020	valid-mae:0.14105
[50]	train-mae:0.11193	valid-mae:0.11260
[60]	train-mae:0.09394	valid-mae:0.09453
[70]	train-mae:0.08293	valid-mae:0.08344
[80]	train-mae:0.07642	valid-mae:0.07695
[90]	train-mae:0.07269	valid-mae:0.07325
[100]	train-mae:0.07057	valid-mae:0.07115
[110]	train-mae:0.06938	valid-mae:0.06996
[120]	train-mae:0.06868	valid-mae:0.06931
[130]	train-mae:0.06825	valid-mae:0.06891
[140]	train-mae:0.06800	valid-mae:0.06869
[150]	train-mae:0.06784	valid-mae:0.06855
[160]	train-mae:0.06772	valid-mae:0.06847
[170]	train-mae:0.06763	valid-mae:0.06842
[180]	train-mae:0.06758	valid-mae:0.06841
[190]	trai

In [ ]:
#Predicting the results
Predicted_test_xgb = model_xgb.predict(dtest)

In [ ]:
sample_file = pd.read_csv(opt.input_dir + '/sample_submission.csv')
for c in sample_file.columns[sample_file.columns != 'ParcelId']:
    sample_file[c] = Predicted_test_xgb

print('Preparing the csv file …')
sample_file.to_csv(opt.input_dir + 'xgb_predicted_results.csv', index=False, float_format='%.4f')
print("Finished writing the file")

Preparing the csv file …
Finished writing the file


In [ ]:
Predicted_test_xgb[0]

0.8735636

In [ ]:
sample_file.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0.874,0.874,0.874,0.874,0.874,0.874
1,10759547,0.330,0.330,0.330,0.330,0.330,0.330
2,10843547,0.184,0.184,0.184,0.184,0.184,0.184
3,10859147,0.060,0.060,0.060,0.060,0.060,0.060
4,10879947,-0.014,-0.014,-0.014,-0.014,-0.014,-0.014
